In [21]:
import cv2
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing import image as img
import numpy as np

In [22]:
model=model_from_json(open('fin.json','r').read())
model.load_weights('fe.h5')

In [23]:
face_cls=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap=cv2.VideoCapture(0)
ma={1:'Mask',0:'No Mask'}
color={1:(0,255,0),0:(0,0,255)}
color[0]

(0, 0, 255)

In [24]:
while True:
    ret,frame=cap.read()
    if not ret:
        continue
    grey_img=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    face=face_cls.detectMultiScale(grey_img,1.3,5)
    for (x,y,w,h) in face:
        cropped_img=frame[x:x+w,y:y+h]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        grey_cropped=cv2.cvtColor(cropped_img,cv2.COLOR_BGR2GRAY)
        resized=cv2.resize(grey_cropped,(100,100))
        samples=img.img_to_array(resized)
        samples=np.array(samples/255.0,'float32').reshape(1,100,100,1)
        prediction=model.predict(samples)
        index=np.argmax(prediction[0])
        cv2.putText(frame,ma[index]+' '+str(int(np.round(prediction[0][index]*100)))+' %',(x,y-9),cv2.FONT_ITALIC,0.5,color[index],1)
        cv2.imshow('face',frame)
        #print(prediction)
    if cv2.waitKey(10)==13:
        break
cap.release()
cv2.destroyAllWindows()